# DS 201 C*
See DataStax Academy course here: https://academy.datastax.com/resources/ds201-foundations-apache-cassandra

In [ ]:
!pip install cassandra-driver
!pip install names
!pip install RandomWords

In [ ]:
from cassandra.cluster import Cluster
import uuid
import names
from random_words import RandomWords
import random
import string

Connect to the cluster. The cluster should be run using Docker with the following:
```
docker run -p 9042:9042 -p 9160:9160 --name cassandra -d cassandra:latest
```

In [ ]:
# cluster = Cluster(['127.0.0.1'])
cluster = Cluster(['cassandra-node'])
session = cluster.connect()

Generate the `killrvideo` keyspace

In [ ]:
session.execute(
    """
    CREATE KEYSPACE killrvideo WITH REPLICATION = {
        'class': 'SimpleStrategy',
        'replication_factor': 1
    };
    """
)

Set keyspace to `killrvideo` keyspace

In [ ]:
session.set_keyspace('killrvideo')

Generate tables

In [ ]:
session.execute(
    """
    CREATE TABLE movies_by_actor (
        actor TEXT,
        release_year INT,
        movie_id UUID,
        title TEXT,
        genres set<TEXT>,
        rating FLOAT,
        PRIMARY KEY ((actor), release_year, movie_id)
        ) WITH CLUSTERING ORDER BY (release_year DESC, movie_id ASC);
    """
)

Add some data to our table

In [ ]:
def gen_actor():
    rw = RandomWords()
    letter = random.choice('qwertyuiopasdfghjklzcvbnm')
    genres = ['Action','Adventure','Comedy','Crime','Drama','Fantasy','Historical','Horror','Mystery','Romance','SciFi']
    num_genres = random.randint(1,3)
    
    actor = names.get_full_name()
    release_year = random.randint(1970,2018)
    movie_id = uuid.uuid1()
    title = ' '.join(rw.random_words(letter=letter, count=3)).title()
    genres = set(random.sample(genres, num_genres))
    rating = random.randint(1,5)
    
    return (actor, release_year, movie_id, title, genres, rating)    

In [ ]:
for i in range(1000):
    actor = gen_actor()
    
    session.execute(
        """
        INSERT INTO movies_by_actor (actor, release_year, movie_id, title, genres, rating)
        VALUES (%s, %s, %s, %s, %s, %s)
        """,
        actor
    )    

We can query our keyspace to demonstrate that the data entered the DB:

In [ ]:
results = session.execute(
    """
    select * from movies_by_actor limit 10
    """
)

for row in results:
    print(row)